<a href="https://colab.research.google.com/github/keshavkmr48/Kaggle/blob/master/NLP_DISASTER_TWEET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting up kaggle API to download competition dataset**

In [28]:
# !pip install kaggle --upgrade
!pip install transformers

! mkdir .kaggle
import json
token = {"username":"keshavkmr48","key":"014d22aa9430a95daaf028ec3e3463a2"}
with open('/content/.kaggle/kaggle.json','w') as file:
  json.dump(token,file)
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
# download competition data
!kaggle competitions download -c nlp-getting-started

- path is now set to: {/content}
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 40.8MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 55.8MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.6MB/s]


# **Loading Libraries**

In [29]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance
import tensorflow as tf
from tqdm import tqdm_notebook
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import torch
import transformers as ppb
import nltk
nltk.download('stopwords')
nltk.download('punkt')
STOPWORDS = set(stopwords.words('english'))

%matplotlib inline

from sklearn import ensemble
# from catboost import CatBoostRegressor
from matplotlib import pyplot
# import shap
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import gc, os
import json
import string,re
from bs4 import BeautifulSoup
from collections import defaultdict
from wordcloud import STOPWORDS
from sklearn import preprocessing
from sklearn import metrics
import joblib
pd.set_option('display.max_columns', 1000)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# ***Reading all tain,test and submission data file***

In [30]:
# list data in the downloaded folder
#os.listdir('{/content}/competitions/data-science-bowl-2019')
os.chdir('{/content}/competitions/nlp-getting-started')
os.listdir()

['sample_submission.csv', 'test.csv', 'train.csv']

In [0]:
def read_data():
    print('Reading train.csv file....')
    train = pd.read_csv('train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

    print('Reading test.csv file....')
    test = pd.read_csv('test.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))


    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv('sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, sample_submission

In [32]:
train, test, sample_submission = read_data()


Reading train.csv file....
Training.csv file have 7613 rows and 5 columns
Reading test.csv file....
Test.csv file have 3263 rows and 4 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 3263 rows and 2 columns


In [0]:
# from multiprocessing import Pool

# def json_loader(x):
#   return json.loads(x)
# pool = Pool(4)
# result = pool.map_async(json_loader,train.event_data)
# df = pd.DataFrame(list(result.get()))

# ***Peak into data***

In [34]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [0]:
train.text = train.text.astype('str')
test.text = test.text.astype('str')

# ***Slang , Misspelling and Stopword dictionary***

In [0]:
slang_abbrev_dict = {
    'AFAIK': 'As Far As I Know','AFK': 'Away From Keyboard','ASAP': 'As Soon As Possible','ATK': 'At The Keyboard','ATM': 'At The Moment','A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard','BBL': 'Be Back Later','BBS': 'Be Back Soon','BFN': 'Bye For Now','B4N': 'Bye For Now','BRB': 'Be Right Back','BRT': 'Be Right There',
    'BTW': 'By The Way','B4': 'Before','B4N': 'Bye For Now','CU': 'See You','CUL8R': 'See You Later','CYA': 'See You','FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed','FWIW': 'For What It\'s Worth','FYI': 'For Your Information','GAL': 'Get A Life','GG': 'Good Game','GN': 'Good Night','GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!','G9': 'Genius','IC': 'I See','ICQ': 'I Seek you','ILU': 'I Love You','IMHO': 'In My Humble Opinion','IMO': 'In My Opinion','IOW': 'In Other Words',
    'IRL': 'In Real Life','KISS': 'Keep It Simple, Stupid','LDR': 'Long Distance Relationship','LMAO': 'Laugh My Ass Off','LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See','L8R': 'Later','MTE': 'My Thoughts Exactly','M8': 'Mate','NRN': 'No Reply Necessary','OIC': 'Oh I See','OMG': 'Oh My God',
    'PITA': 'Pain In The Ass','PRT': 'Party','PRW': 'Parents Are Watching','QPSA?': 'Que Pasa?','ROFL': 'Rolling On The Floor Laughing','ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My Ass Off','SK8': 'Skate','STATS': 'Your sex and age','ASL': 'Age, Sex, Location','THX': 'Thank You','TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later','U': 'You','U2': 'You Too','U4E': 'Yours For Ever','WB': 'Welcome Back','WTF': 'What The Fuck','WTG': 'Way To Go!',
    'WUF': 'Where Are You From?','W8': 'Wait','7K': 'Sick:-D Laugher'}

In [0]:
mispell_dict = {"aren't" : "are not","can't" : "cannot","couldn't" : "could not","couldnt" : "could not","didn't" : "did not","doesn't" : "does not",
"doesnt" : "does not","don't" : "do not","hadn't" : "had not","hasn't" : "has not","haven't" : "have not","havent" : "have not","he'd" : "he would",
"he'll" : "he will","he's" : "he is","i'd" : "I would","i'd" : "I had","i'll" : "I will","i'm" : "I am","isn't" : "is not","it's" : "it is",
"it'll":"it will","i've" : "I have","let's" : "let us","mightn't" : "might not","mustn't" : "must not","shan't" : "shall not","she'd" : "she would",
"she'll" : "she will","she's" : "she is","shouldn't" : "should not","shouldnt" : "should not","that's" : "that is","thats" : "that is","there's" : "there is",
"theres" : "there is","they'd" : "they would","they'll" : "they will","they're" : "they are","theyre":  "they are","they've" : "they have","we'd" : "we would",
"we're" : "we are","weren't" : "were not","we've" : "we have","what'll" : "what will","what're" : "what are","what's" : "what is","what've" : "what have",
"where's" : "where is","who'd" : "who would","who'll" : "who will","who're" : "who are","who's" : "who is","who've" : "who have","won't" : "will not",
"wouldn't" : "would not","you'd" : "you would","you'll" : "you will","you're" : "you are","you've" : "you have","'re": " are","wasn't": "was not",
"we'll":" will","didn't": "did not","tryin'":"trying"}

In [0]:
    add_stopwords=["a", "about", "above", "after", "again", "against", "ain", "all", "am","an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because",
    "been", "before", "being", "below", "between", "both", "but", "by", "can","couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn","doesn't", 
    "doing", "don", "don't", "down", "during", "each", "few", "for","from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have",
    "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him","himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't",
    "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn","mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn",
    "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once","only", "or", "other", "our", "ours", "ourselves", "out", "over", "own",
    "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've","shouldn", "shouldn't", "so", "some", "such", "t", "than", "that",
    "that'll", "the", "their", "theirs", "them", "themselves", "then", "there","these", "they", "this", "those", "through", "to", "too", "under", "until",
    "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren","weren't", "what", "when", "where", "which", "while", "who", "whom", "why",
    "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd","you'll", "you're", "you've", "your", "yours", "yourself", "yourselves",
    "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm","i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd",
    "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've","what's", "when's", "where's", "who's", "why's", "would", "able", "abst",
    "accordance", "according", "accordingly", "across", "act", "actually","added", "adj", "affected", "affecting", "affects", "afterwards", "ah",
    "almost", "alone", "along", "already", "also", "although", "always","among", "amongst", "announce", "another", "anybody", "anyhow", "anymore",
    "anyone", "anything", "anyway", "anyways", "anywhere", "apparently","approximately", "arent", "arise", "around", "aside", "ask", "asking",
    "auth", "available", "away", "awfully", "b", "back", "became", "become","becomes", "becoming", "beforehand", "begin", "beginning", "beginnings",
    "begins", "behind", "believe", "beside", "besides", "beyond", "biol","brief", "briefly", "c", "ca", "came", "cannot", "can't", "cause", "causes",
    "certain", "certainly", "co", "com", "come", "comes", "contain","containing", "contains", "couldnt", "date", "different", "done",
    "downwards", "due", "e", "ed", "edu", "effect", "eg", "eight", "eighty","either", "else", "elsewhere", "end", "ending", "enough", "especially",
    "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything","everywhere", "ex", "except", "f", "far", "ff", "fifth", "first", "five",
    "fix", "followed", "following", "follows", "former", "formerly", "forth","found", "four", "furthermore", "g", "gave", "get", "gets", "getting",
    "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten","h", "happens", "hardly", "hed", "hence", "hereafter", "hereby", "herein",
    "heres", "hereupon", "hes", "hi", "hid", "hither", "home", "howbeit","however", "hundred", "id", "ie", "im", "immediate", "immediately",
    "importance", "important", "inc", "indeed", "index", "information","instead", "invention", "inward", "itd", "it'll", "j", "k", "keep", "keeps",
    "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last","lately", "later", "latter", "latterly", "least", "less", "lest", "let",
    "lets", "like", "liked", "likely", "line", "little", "'ll", "look","looking", "looks", "ltd", "made", "mainly", "make", "makes", "many", "may",
    "maybe", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might","million", "miss", "ml", "moreover", "mostly", "mr", "mrs", "much", "mug",
    "must", "n", "na", "name", "namely", "nay", "nd", "near", "nearly","necessarily", "necessary", "need", "needs", "neither", "never",
    "nevertheless", "new", "next", "nine", "ninety", "nobody", "non", "none","nonetheless", "noone", "normally", "nos", "noted", "nothing", "nowhere",
    "obtain", "obtained", "obviously", "often", "oh", "ok", "okay", "old","omitted", "one", "ones", "onto", "ord", "others", "otherwise", "outside",
    "overall", "owing", "p", "page", "pages", "part", "particular","particularly", "past", "per", "perhaps", "placed", "please", "plus",
    "poorly", "possible", "possibly", "potentially", "pp", "predominantly","present", "previously", "primarily", "probably", "promptly", "proud",
    "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran","rather", "rd", "readily", "really", "recent", "recently", "ref", "refs",
    "regarding", "regardless", "regards", "related", "relatively", "research","respectively", "resulted", "resulting", "results", "right", "run", "said",
    "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem","seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven",
    "several", "shall", "shed", "shes", "show", "showed", "shown", "showns","shows", "significant", "significantly", "similar", "similarly", "since",
    "six", "slightly", "somebody", "somehow", "someone", "somethan","something", "sometime", "sometimes", "somewhat", "somewhere", "soon",
    "sorry", "specifically", "specified", "specify", "specifying", "still","stop", "strongly", "sub", "substantially", "successfully", "sufficiently",
    "suggest", "sup", "sure", "take", "taken", "taking", "tell", "tends", "th","thank", "thanks", "thanx", "thats", "that've", "thence", "thereafter",
    "thereby", "thered", "therefore", "therein", "there'll", "thereof","therere", "theres", "thereto", "thereupon", "there've", "theyd", "theyre",
    "think", "thou", "though", "thoughh", "thousand", "throug", "throughout","thru", "thus", "til", "tip", "together", "took", "toward", "towards",
    "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un","unfortunately", "unless", "unlike", "unlikely", "unto", "upon", "ups",
    "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using","usually", "v", "value", "various", "'ve", "via", "viz", "vol", "vols",
    "vs", "w", "want", "wants", "wasnt", "way", "wed", "welcome", "went","werent", "whatever", "what'll", "whats", "whence", "whenever",
    "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon","wherever", "whether", "whim", "whither", "whod", "whoever", "whole",
    "who'll", "whomever", "whos", "whose", "widely", "willing", "wish","within", "without", "wont", "words", "world", "wouldnt", "www", "x", "yes",
    "yet", "youd", "youre", "z", "zero", "a's", "ain't", "allow", "allows","apart", "appear", "appreciate", "appropriate", "associated", "best",
    "better", "c'mon", "c's", "cant", "changes", "clearly", "concerning","consequently", "consider", "considering", "corresponding", "course",
    "currently", "definitely", "described", "despite", "entirely", "exactly","example", "going", "greetings", "hello", "help", "hopefully", "ignored",
    "inasmuch", "indicate", "indicated", "indicates", "inner", "insofar","it'd", "keep", "keeps", "novel", "presumably", "reasonably", "second",
    "secondly", "sensible", "serious", "seriously", "sure", "t's", "third","thorough", "thoroughly", "three", "well", "wonder", "a", "about", "above",
    "above", "across", "after", "afterwards", "again", "against", "all","almost", "alone", "along", "already", "also", "although", "always", "am",
    "among", "amongst", "amoungst", "amount", "an", "and", "another", "any","anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as",
    "at", "back", "be", "became", "because", "become", "becomes", "becoming","been", "before", "beforehand", "behind", "being", "below", "beside",
    "besides", "between", "beyond", "bill", "both", "bottom", "but", "by","call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry",
    "de", "describe", "detail", "do", "done", "down", "due", "during", "each","eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough",
    "etc", "even", "ever", "every", "everyone", "everything", "everywhere","except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five",
    "for", "former", "formerly", "forty", "found", "four", "from", "front","full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he",
    "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers","herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if",
    "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself","keep", "last", "latter", "latterly", "least", "less", "ltd", "made",
    "many", "may", "me", "meanwhile", "might", "mill", "mine", "more","moreover", "most", "mostly", "move", "much", "must", "my", "myself",
    "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no","nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of",
    "off", "often", "on", "once", "one", "only", "onto", "or", "other","others", "otherwise", "our", "ours", "ourselves", "out", "over", "own",
    "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see","seem", "seemed", "seeming", "seems", "serious", "several", "she", "should",
    "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow","someone", "something", "sometime", "sometimes", "somewhere", "still",
    "such", "system", "take", "ten", "than", "that", "the", "their", "them","themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thickv", "thin","third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards","twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when","whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither","who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within",
    "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves","the", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n",
    "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "A", "B", "C","D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R",
    "S", "T", "U", "V", "W", "X", "Y", "Z", "co", "op", "research-articl","pagecount", "cit", "ibid", "les", "le", "au", "que", "est", "pas", "vol",
    "el", "los", "pp", "u201d", "well-b", "http", "volumtype", "par", "0o","0s", "3a", "3b", "3d", "6b", "6o", "a1", "a2", "a3", "a4", "ab", "ac",
    "ad", "ae", "af", "ag", "aj", "al", "an", "ao", "ap", "ar", "av", "aw","ax", "ay", "az", "b1", "b2", "b3", "ba", "bc", "bd", "be", "bi", "bj",
    "bk", "bl", "bn", "bp", "br", "bs", "bt", "bu", "bx", "c1", "c2", "c3","cc", "cd", "ce", "cf", "cg", "ch", "ci", "cj", "cl", "cm", "cn", "cp",
    "cq", "cr", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d2", "da", "dc","dd", "de", "df", "di", "dj", "dk", "dl", "do", "dp", "dr", "ds", "dt",
    "du", "dx", "dy", "e2", "e3", "ea", "ec", "ed", "ee", "ef", "ei", "ej","el", "em", "en", "eo", "ep", "eq", "er", "es", "et", "eu", "ev", "ex",
    "ey", "f2", "fa", "fc", "ff", "fi", "fj", "fl", "fn", "fo", "fr", "fs", "ft", "fu", "fy", "ga", "ge", "gi", "gj", "gl", "go", "gr", "gs", "gy",
    "h2", "h3", "hh", "hi", "hj", "ho", "hr", "hs", "hu", "hy", "i", "i2", "i3","i4", "i6", "i7", "i8", "ia", "ib", "ic", "ie", "ig", "ih", "ii", "ij","il", "in", "io", "ip", "iq", "ir", "iv", "ix", "iy", "iz", "jj", "jr",
    "js", "jt", "ju", "ke", "kg", "kj", "km", "ko", "l2", "la", "lb", "lc","lf", "lj", "ln", "lo", "lr", "ls", "lt", "m2", "ml", "mn", "mo", "ms",
    "mt", "mu", "n2", "nc", "nd", "ne", "ng", "ni", "nj", "nl", "nn", "nr","ns", "nt", "ny", "oa", "ob", "oc", "od", "of", "og", "oi", "oj", "ol",
    "om", "on", "oo", "oq", "or", "os", "ot", "ou", "ow", "ox", "oz", "p1","p2", "p3", "pc", "pd", "pe", "pf", "ph", "pi", "pj", "pk", "pl", "pm",
    "pn", "po", "pq", "pr", "ps", "pt", "pu", "py", "qj", "qu", "r2", "ra","rc", "rd", "rf", "rh", "ri", "rj", "rl", "rm", "rn", "ro", "rq", "rr",
    "rs", "rt", "ru", "rv", "ry", "s2", "sa", "sc", "sd", "se", "sf", "si","sj", "sl", "sm", "sn", "sp", "sq", "sr", "ss", "st", "sy", "sz", "t1",
    "t2", "t3", "tb", "tc", "td", "te", "tf", "th", "ti", "tj", "tl", "tm","tn", "tp", "tq", "tr", "ts", "tt", "tv", "tx", "ue", "ui", "uj", "uk",
    "um", "un", "uo", "ur", "ut", "va", "wa", "vd", "wi", "vj", "vo", "wo","vq", "vt", "vu", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn",
    "xo", "xs", "xt", "xv", "xx", "y2", "yj", "yl", "yr", "ys", "yt", "zi", "zz"]

# ***Text data cleaning framework***

In [0]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [0]:
def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [0]:
def clean_text(x):
    x = str(x).replace("\n","")
    
    stops  = set(list(STOPWORDS)+add_stopwords)
    text = [w for w in word_tokenize(x) if w not in stops]    
    text = " ".join(text)
    
    return text

In [0]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

In [0]:
def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

In [0]:
def unslang(text):
    """Converts text like "OMG" into "Oh my God"
    """
    text = [slang_abbrev_dict[w.upper()] if w.upper() in slang_abbrev_dict.keys() else w for w in word_tokenize(text)]    
    return " ".join(text)

In [0]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [0]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [0]:
def remove_html(text):
    return BeautifulSoup(text, "lxml").text

In [0]:
def clean_data(df, col):
        df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: remove_urls(x))
        df[col] = df[col].apply(lambda x: remove_html(x))
        df[col] = df[col].apply(lambda text: remove_punctuation(text))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))
        df[col] = df[col].apply(lambda x: unslang(x))
        df[col] = df[col].apply(lambda x: remove_emoji(x))
        return df

In [0]:
train = clean_data(train, 'text')
test = clean_data(test, 'text')

# ***Creating k-Folds (k=5) for training data***

In [50]:
if __name__ == "__main__":
    train["kfold"] = -1

    train = train.sample(frac=1).reset_index(drop=True)

    kf = StratifiedKFold(n_splits=5, shuffle=False, random_state=42)


    for fold, (train_idx, val_idx) in enumerate(kf.split(X=train, y=train.target.values)):
        print(len(train_idx), len(val_idx))
        train.loc[val_idx, 'kfold'] = fold
    

    train.to_csv("train_folds.csv", index=False)
del train
TRAINING_DATA = pd.read_csv('train_folds.csv')

6089 1524
6090 1523
6091 1522
6091 1522
6091 1522


# ***Missing Value Imputation***

In [0]:
# Missing value filler 
TRAINING_DATA.keyword = TRAINING_DATA.keyword.fillna('No_Location')
TRAINING_DATA.location = TRAINING_DATA.location.fillna('No_Location')
test.keyword = test.keyword.fillna('No_Key')
test.location = test.location.fillna('No_Location')

# ***n-gram insights***

In [0]:
# n-gram generator
def generate_ngrams(text, n_gram=1):
  try:
    token = [token for token in text.split(' ') if token != '' if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]
  except:
    print(text)

def n_gram_distribution_generator(n_gram=1):
  disaster_ngrams = defaultdict(int)
  nondisaster_ngrams = defaultdict(int)

  for tweet in TRAINING_DATA.text[TRAINING_DATA.target==1]:
      for word in generate_ngrams(tweet,n_gram=n_gram):
          disaster_ngrams[word] += 1

  for tweet in TRAINING_DATA.text[TRAINING_DATA.target==0]:
      for word in generate_ngrams(tweet,n_gram=n_gram):
          nondisaster_ngrams[word] += 1
  df_disaster_ngrams= pd.DataFrame(disaster_ngrams.items(), columns=['word','count']).sort_values(by='count', ascending=False)
  df_non_disaster_ngrams = pd.DataFrame(nondisaster_ngrams.items(), columns=['word','count']).sort_values(by='count', ascending=False)
  return df_disaster_ngrams, df_non_disaster_ngrams

In [0]:
TRAINING_DATA=TRAINING_DATA[-TRAINING_DATA.text.isnull()]

UNIGRAM

In [55]:
df_disaster_ngrams, df_non_disaster_ngrams = n_gram_distribution_generator(n_gram=1)
print(f'frequency count of unigram for disaster tweets \n {df_disaster_ngrams.head()}');
print( f'frequency count of unigram for non-disaster tweets \n {df_non_disaster_ngrams.head()}')

frequency count of unigram for disaster tweets 
            word  count
58         news    136
254    disaster    117
81   california    111
397     suicide    110
31       police    106
frequency count of unigram for non-disaster tweets 
        word  count
21     dont    139
16     body    112
230   video     96
163  people     92
170       2     92


BIGRAM

In [56]:
df_disaster_ngrams, df_non_disaster_ngrams = n_gram_distribution_generator(n_gram=2)
print(f'frequency count of bigram for disaster tweets \n {df_disaster_ngrams.head()}');
print( f'frequency count of bigram for non-disaster tweets \n {df_non_disaster_ngrams.head()}')

frequency count of bigram for disaster tweets 
                      word  count
379        suicide bomber     59
72    northern california     41
1204            oil spill     38
822     burning buildings     35
73    california wildfire     34
frequency count of bigram for non-disaster tweets 
               word  count
818       Out Loud     60
817   Laughing Out     60
13      cross body     38
381  youtube video     36
14        body bag     26


TRIGRAM

In [57]:
df_disaster_ngrams, df_non_disaster_ngrams = n_gram_distribution_generator(n_gram=3)
print(f'frequency count of trigram for disaster tweets \n {df_disaster_ngrams.head()}');
print( f'frequency count of trigram for non-disaster tweets \n {df_non_disaster_ngrams.head()}')

frequency count of trigram for disaster tweets 
                               word  count
1708      suicide bomber detonated     30
63    northern california wildfire     29
60              latest homes razed     28
61            homes razed northern     28
2458         bomber detonated bomb     28
frequency count of trigram for non-disaster tweets 
                              word  count
699             Laughing Out Loud     60
459   reddit quarantine offensive     18
460  quarantine offensive content     18
11                 cross body bag     18
137                 pick fan army     17


# ***Feature Generation***

In [58]:
# feature_engineering on text column

# char_count
TRAINING_DATA['char_count'] = TRAINING_DATA['text'].apply(lambda x: len(str(x)))
test['char_count'] = test['text'].apply(lambda x: len(str(x)))

# punctuation_count
TRAINING_DATA['punctuation_count'] = TRAINING_DATA['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test['punctuation_count'] = test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
TRAINING_DATA['hashtag_count'] = TRAINING_DATA['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
test['hashtag_count'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
TRAINING_DATA['mention_count'] = TRAINING_DATA['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
test['mention_count'] = test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

# unique_word_count
TRAINING_DATA['unique_word_count'] = TRAINING_DATA['text'].apply(lambda x: len(set(str(x).split())))
test['unique_word_count'] = test['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
TRAINING_DATA['stop_word_count'] = TRAINING_DATA['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
test['stop_word_count'] = test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

# url_count
TRAINING_DATA['url_count'] = TRAINING_DATA['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test['url_count'] = test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
TRAINING_DATA['mean_word_length'] = TRAINING_DATA['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test['mean_word_length'] = test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
# word_count
TRAINING_DATA['word_count'] = TRAINING_DATA['text'].apply(lambda x: len(str(x).split()))
test['word_count'] = test['text'].apply(lambda x: len(str(x).split()))

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [59]:
TRAINING_DATA.head(3)

,id,keyword,location,text,target,kfold,char_count,punctuation_count,hashtag_count,mention_count,unique_word_count,stop_word_count,url_count,mean_word_length,word_count
0,4173,drown,"Lynwood, CA",gon drown mustard lemon pepper,0,0,30,0,0,0,5,0,0,5.200000,5
1,8783,siren,"Honolulu,Hawaii",serephina siren 3,0,0,17,0,0,0,3,0,0,5.000000,3
2,5689,floods,Philippines,pagasa 7am yellow warning panay island guimara...,1,0,105,0,0,0,13,0,0,6.571429,14


In [60]:
TRAINING_DATA.text.head()

0                       gon drown mustard lemon pepper
1                                    serephina siren 3
2    pagasa 7am yellow warning panay island guimara...
3    criminals hijack lorries buses arrested enugu ...
4              critters climate plague deaths colorado
Name: text, dtype: object

## ***text embedding using BERT***

In [0]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [0]:
def fetch_vectors(string_list, batch_size=64):
    # inspired by https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
    DEVICE = torch.device("cuda")
    
    # For DistilBERT:
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
    
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)
    
    model.to(DEVICE)

    fin_features = []
    for data in tqdm_notebook(chunks(string_list, batch_size)):
        tokenized = []
        for x in data:
            x = " ".join(x.strip().split()[:200])
            tok = tokenizer.encode(x, add_special_tokens=True)
            tokenized.append(tok[:512])

        max_len = 512
        padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized])
        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded).to(DEVICE)
        attention_mask = torch.tensor(attention_mask).to(DEVICE)

        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask)
        
        features = last_hidden_states[0][:, 0, :].cpu().numpy()
        fin_features.append(features)

    fin_features = np.vstack(fin_features)
    return fin_features,tokenizer

In [63]:
gc.collect()

train_dense,traintokenizer = fetch_vectors(TRAINING_DATA.text.values)
test_dense,testtokenizer = fetch_vectors(test.text.values)

In [0]:
assert(len(train_dense)==TRAINING_DATA.shape[0])
assert(len(test_dense)==test.shape[0])

In [65]:
print(train_dense[:3]); print(test_dense[:3])

[[-0.26678988  0.07835416 -0.04069579 ... -0.13735893  0.18777059
   0.3241973 ]
 [-0.2644404  -0.10522754 -0.13312659 ... -0.08269488  0.29187024
   0.32950544]
 [-0.4256857  -0.01215183 -0.05950172 ... -0.09845211  0.34118354
   0.01000265]]
[[-0.25721332 -0.15935762 -0.13349877 ... -0.33479208  0.27146503
   0.08463379]
 [-0.02239156 -0.19096556  0.01433134 ... -0.12936781  0.19693802
   0.09040565]
 [-0.28844002  0.01489966 -0.1840436  ... -0.00638565  0.11386836
   0.319541  ]]


In [66]:
print(traintokenizer); print(testtokenizer)

In [67]:
TRAINING_DATA.head()

,id,keyword,location,text,target,kfold,char_count,punctuation_count,hashtag_count,mention_count,unique_word_count,stop_word_count,url_count,mean_word_length,word_count
0,4173,drown,"Lynwood, CA",gon drown mustard lemon pepper,0,0,30,0,0,0,5,0,0,5.200000,5
1,8783,siren,"Honolulu,Hawaii",serephina siren 3,0,0,17,0,0,0,3,0,0,5.000000,3
2,5689,floods,Philippines,pagasa 7am yellow warning panay island guimara...,1,0,105,0,0,0,13,0,0,6.571429,14
3,6150,hijack,Nigeria,criminals hijack lorries buses arrested enugu ...,1,0,51,0,0,0,7,0,0,6.428571,7
4,3095,deaths,"Washington, D.C.",critters climate plague deaths colorado,1,0,39,0,0,0,5,0,0,7.000000,5


In [68]:
train_dense_temp = pd.DataFrame(train_dense)
train_dense_temp.shape

(7608, 768)

In [69]:
print(TRAINING_DATA.shape)
print(train_dense_temp.shape)

(7608, 15)
(7608, 768)


In [70]:
train_birt = pd.concat([TRAINING_DATA.reset_index(drop=True),train_dense_temp.reset_index(drop=True)],axis=1)
train_birt.shape

(7608, 783)

In [71]:
test_dense_temp = pd.DataFrame(test_dense)
test_dense_temp.shape

(3263, 768)

In [72]:
test_birt = pd.concat([test.reset_index(drop=True),test_dense_temp.reset_index(drop=True)],axis=1)
test_birt.shape

(3263, 781)

In [73]:
set(TRAINING_DATA.columns)-set(test.columns)

{'kfold', 'target'}

# ***Feature Engineering***

In [75]:
train_birt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7608 entries, 0 to 7607
Columns: 783 entries, id to 767
dtypes: float32(768), float64(1), int64(11), object(3)
memory usage: 23.2+ MB


In [76]:
test_birt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Columns: 781 entries, id to 767
dtypes: float32(768), float64(1), int64(9), object(3)
memory usage: 9.9+ MB


## ***For Tree Models***

### ***Numerical Features***

### ***Categorical Features***

## ***For Linear & Neural Network Models***

In [0]:
# MODELS = {
#     "randomforest": ensemble.RandomForestClassifier(n_estimators=200, n_jobs=-1, verbose=2),
#     "extratrees": ensemble.ExtraTreesClassifier(n_estimators=200, n_jobs=-1, verbose=2),
# }


In [0]:
# FOLD_MAPPPING = {
#     0: [1, 2, 3, 4],
#     1: [0, 2, 3, 4],
#     2: [0, 1, 3, 4],
#     3: [0, 1, 2, 4],
#     4: [0, 1, 2, 3]
# }

In [0]:
# if __name__ == "__main__":

#   for MODEL in MODELS.keys():
#     for FOLD in range(5):
#       train_df = TRAINING_DATA[TRAINING_DATA.kfold.isin(FOLD_MAPPPING.get(FOLD))].reset_index(drop=True)
#       valid_df = TRAINING_DATA[TRAINING_DATA.kfold==FOLD].reset_index(drop=True)

#       ytrain = train_df.target.values
#       yvalid = valid_df.target.values

#       train_df = train_df.drop(["id", "target", "kfold"], axis=1)
#       valid_df = valid_df.drop(["id", "target", "kfold"], axis=1)

#       train_df.text = train_df.text.apply(lambda x : text_processor(x))
#       valid_df.text = valid_df.text.apply(lambda x : text_processor(x))

#       valid_df = valid_df[train_df.columns]

#       # label_encoders = {}
#       # for c in train_df.columns:
#       #     lbl = preprocessing.LabelEncoder()
#       #     lbl.fit(train_df[c].values.tolist() + valid_df[c].values.tolist() + df_test[c].values.tolist())
#       #     train_df.loc[:, c] = lbl.transform(train_df[c].values.tolist())
#       #     valid_df.loc[:, c] = lbl.transform(valid_df[c].values.tolist())
#       #     label_encoders[c] = lbl
      
#       # data is ready to train
#       clf = MODELS[MODEL]
#       clf.fit(train_df, ytrain)
#       preds = clf.predict_proba(valid_df)[:, 1]
#       print(metrics.roc_auc_score(yvalid, preds))

#       # joblib.dump(label_encoders, f"models/{MODEL}_{FOLD}_label_encoder.pkl")
#       joblib.dump(clf, f"models/{MODEL}_{FOLD}.pkl")
#       joblib.dump(train_df.columns, f"models/{MODEL}_{FOLD}_columns.pkl")
